## Decorators

A decorator is a callable that takes another function as an argument (the decorated
 function).
 A decorator may perform some processing with the decorated function, and returns
 it or replaces it with another function or callable object.

Python does not require you to declare variables, but assumes that a variable assigned in the body of a function is local. That's why you use `global` keyword when you wanna use a global var in a local scope.

function decorators are executed
as soon as the module is imported, but the decorated functions only run when they
are explicitly invoked. This highlights the difference between what Pythonistas call
import time and runtime.

In [3]:
registry = []
def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
if __name__ == '__main__':
    main()

running register(<function f1 at 0x72c7c5570220>)
running register(<function f2 at 0x72c7c5523e20>)
running main()
registry -> [<function f1 at 0x72c7c5570220>, <function f2 at 0x72c7c5523e20>]
running f1()
running f2()
running f3()


## Cache and lre_cache

The functools.cache decorator -> implements memoization
it saves the result of previous invocations of an expensive func.

All the arguments taken by the decorated func must be hashable cause the cache users a dict to store the results.

The main advantage of @lru_cache is that its memory usage is bounded by the maxsize param.

In [ ]:
import functools

@functools.cache
def fibonacci(n):
    return 1 if n < 2 else fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

13


A more real example related to API rate-limited calls:

In [12]:
import functools
import requests

@functools.lru_cache(maxsize=100)
def get_country_info(country_code):
    print(f"Fetching info for {country_code}")
    response = requests.get(f"https://restcountries.com/v3.1/alpha/{country_code}")
    return response.json()

# Usage
print(get_country_info("us") ) # Hits API


Fetching info for us
[{'name': {'common': 'United States', 'official': 'United States of America', 'nativeName': {'eng': {'official': 'United States of America', 'common': 'United States'}}}, 'tld': ['.us'], 'cca2': 'US', 'ccn3': '840', 'cioc': 'USA', 'independent': True, 'status': 'officially-assigned', 'unMember': True, 'currencies': {'USD': {'symbol': '$', 'name': 'United States dollar'}}, 'idd': {'root': '+1', 'suffixes': ['201', '202', '203', '205', '206', '207', '208', '209', '210', '212', '213', '214', '215', '216', '217', '218', '219', '220', '224', '225', '227', '228', '229', '231', '234', '239', '240', '248', '251', '252', '253', '254', '256', '260', '262', '267', '269', '270', '272', '274', '276', '281', '283', '301', '302', '303', '304', '305', '307', '308', '309', '310', '312', '313', '314', '315', '316', '317', '318', '319', '320', '321', '323', '325', '327', '330', '331', '334', '336', '337', '339', '346', '347', '351', '352', '360', '361', '364', '380', '385', '386', '4

In [ ]:
print(get_country_info("us"))  # Returns cached result

[{'name': {'common': 'United States', 'official': 'United States of America', 'nativeName': {'eng': {'official': 'United States of America', 'common': 'United States'}}}, 'tld': ['.us'], 'cca2': 'US', 'ccn3': '840', 'cioc': 'USA', 'independent': True, 'status': 'officially-assigned', 'unMember': True, 'currencies': {'USD': {'symbol': '$', 'name': 'United States dollar'}}, 'idd': {'root': '+1', 'suffixes': ['201', '202', '203', '205', '206', '207', '208', '209', '210', '212', '213', '214', '215', '216', '217', '218', '219', '220', '224', '225', '227', '228', '229', '231', '234', '239', '240', '248', '251', '252', '253', '254', '256', '260', '262', '267', '269', '270', '272', '274', '276', '281', '283', '301', '302', '303', '304', '305', '307', '308', '309', '310', '312', '313', '314', '315', '316', '317', '318', '319', '320', '321', '323', '325', '327', '330', '331', '334', '336', '337', '339', '346', '347', '351', '352', '360', '361', '364', '380', '385', '386', '401', '402', '404', '4

In [14]:
import functools
import pandas as pd

# Load big dataset once
df = pd.DataFrame({
    "id": [1, 2, 3, 4, 5],
    "department": ["sales", "engineering", "sales", "hr", "engineering"]
})

@functools.lru_cache(maxsize=32)
def get_department_employees(department):
    print(f"Filtering department: {department}")
    return df[df["department"] == department]

# Usage
sales = get_department_employees("sales")  # Computes
sales_again = get_department_employees("sales")  # Cached

Filtering department: sales


In [ ]:
from collections import OrderedDict
from functools import wraps

def lru_cache(maxsize=128):
    def decorator(func):
        cache = OrderedDict()

        @wraps(func)
        def wrapper(*args):
            if args in cache:
                cache.move_to_end(args)  
                return cache[args]
            result = func(*args)
            cache[args] = result
            if len(cache) > maxsize:
                cache.popitem(last=False) 
            return result

        return wrapper
    return decorator


## functools.wrap

the original_func becomes a new function (wrapper). But without @wraps, you lose:

__name__

__doc__

__annotations__

__module__


In [ ]:
def decorator(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@decorator
def greet():
    """Say hello"""
    print("Hello")

print(greet.__name__) 
print(greet.__doc__)  

wrapper
None


In [2]:
from functools import wraps

def decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@decorator
def greet():
    """Say hello"""
    print("Hello")

print(greet.__name__)  
print(greet.__doc__)

greet
Say hello


## Function singledispatch
If you decorate a plain func‐
tion with @singledispatch, it becomes the entry point for a generic function: a group
of functions to perform the same operation in different ways, depending on the type
of the first argument.

In [1]:
from functools import singledispatch
import json

@singledispatch
def log_data(data):
    raise NotImplementedError("Unsupported data type")

@log_data.register
def _(data: dict):
    return "Logging JSON: " + json.dumps(data)

@log_data.register
def _(data: str):
    return f"Logging plain string: {data}"

@log_data.register
def _(data: Exception):
    return f"Logging error: {type(data).__name__} - {data}"

# Usage
print(log_data({"user": "Alice", "action": "login"}))
print(log_data("System started"))
print(log_data(ValueError("Invalid input")))


Logging JSON: {"user": "Alice", "action": "login"}
Logging plain string: System started
Logging error: ValueError - Invalid input


## functools.cached_property
Transform a method of a class into a property whose value is computed once and then cached as a normal attribute for the life of the instance. Similar to property(), with the addition of caching.

In [ ]:
from functools import cached_property
import statistics
class DataSet:

    def __init__(self, sequence_of_numbers):
        self._data = tuple(sequence_of_numbers)

    @cached_property
    def stdev(self):
        return statistics.stdev(self._data)

## functools.property
A property object has getter, setter, and deleter methods usable as decorators that create a copy of the property with the corresponding accessor function set to the decorated function.

In [3]:
class House:

    def __init__(self, price):
        self._price = price

    @property
    def price(self):
        return self._price

    @price.setter
    def price(self, new_price):
        if new_price > 0 and isinstance(new_price, float):
            self._price = new_price
        else:
            print("Please enter a valid price")

    @price.deleter
    def price(self):
        del self._price

house = House(50000.0)  # Create instance
house.price = 45000.0   # Update value
house.price 

45000.0